### Import packages

In [1]:
import pandas as pd
import numpy as np

### Fetch data from Snowflake into a file on disk in **feather** format
Click [here](https://www.shaunwang.me/post/post_001_snowpy/) for a quick guide on how to pull data from snowflake by using Python

The following cell are commented out because I only needed to fun it once and I have done that

In [2]:
# from snowpy import run_SQL_to_feather
# run_SQL_to_feather('./SQL','data_pages.sql','pages.feather')
# run_SQL_to_feather('./SQL','data_clicks.sql','clicks.feather')
# run_SQL_to_feather('./SQL','data_v_contact.sql','v_contact.feather')
# print('Done!')

In [3]:
pd.set_option('max_colwidth',None)
pd.set_option('min_rows', 12)

In [4]:
# pd.reset_option("^display")

### Load data of page viewed from feather file into memory and print a summary

In [5]:
pg0 = pd.read_feather('./pages.feather')
pg0.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880922 entries, 0 to 6880921
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   VISITID            6880922 non-null  object        
 1   PAGEID             6880922 non-null  object        
 2   D_SITE             6880922 non-null  category      
 3   PLATFORM           6880922 non-null  object        
 4   CONTACT_UUID_JF    5411591 non-null  object        
 5   CONTACT_UUID_SDK   6880922 non-null  object        
 6   D_DATE_HOUR_VISIT  6880922 non-null  datetime64[ns]
 7   D_DATE_HOUR_EVENT  6880922 non-null  datetime64[ns]
 8   MIGRATION_FLAG     6880922 non-null  category      
 9   PAGE_NAME          6880915 non-null  object        
 10  FIRST_PAGE_NAME    6880922 non-null  object        
 11  END_PAGE_NAME      6880901 non-null  object        
 12  VISIT_LOGGED       6880922 non-null  category      
 13  D_PAGE             6880922 

### Data transformation
The following cell are commented out because I only needed to fun it once and I have done that

In [6]:
# pg0['MIGRATION_FLAG'] = pg0['MIGRATION_FLAG'].astype('category')
# pg0['VISIT_LOGGED'] = pg0['VISIT_LOGGED'].astype('category')
# pg0['D_PAGE_CHAP1'] = pg0['D_PAGE_CHAP1'].astype('category')
# pg0.insert(pg0.columns.get_loc('D_SITE')+1 ,'PLATFORM' \
#     ,pg0['D_SITE'].replace(['.+Android.*', '.+[Ii]OS.*', '.+Web.+'] \
#         ,['Android', 'iOS', 'Web'] \
#         ,regex=True))
# pg0['D_SITE'] = pg0['D_SITE'].astype('category')
# pg0.to_feather('./pages.feather')

### Getting a smaller sample of pages viewed, slicing on date of visit, to be between 2019-Dec-1 and 2020-Jan-31
We don't need 7 million rows for this exercise

In [7]:
pg1 = \
    pg0.pipe(lambda df: df[df['D_DATE_HOUR_VISIT'].between('2019-12-01','2020-01-31')]) \
    .reset_index(drop=True)

### Getting subset of data from the sample in which number of pages viewed per each visit is between 20 and 60

In [8]:
pg1 = pg1.pipe(lambda df: df.loc[df['VISITID'] \
        .isin(df['VISITID'].value_counts().to_frame('PAGE_COUNT') \
            .query("index.str.contains('JF') & PAGE_COUNT.between(20,60)") \
            .index.to_list())]) \
        .reset_index(drop=True).copy()

### Printing a summary of the final sample which we will work on
We have about 120,000 rows

In [9]:
pg1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124344 entries, 0 to 124343
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   VISITID            124344 non-null  object        
 1   PAGEID             124344 non-null  object        
 2   D_SITE             124344 non-null  category      
 3   PLATFORM           124344 non-null  object        
 4   CONTACT_UUID_JF    76891 non-null   object        
 5   CONTACT_UUID_SDK   124344 non-null  object        
 6   D_DATE_HOUR_VISIT  124344 non-null  datetime64[ns]
 7   D_DATE_HOUR_EVENT  124344 non-null  datetime64[ns]
 8   MIGRATION_FLAG     124344 non-null  category      
 9   PAGE_NAME          124342 non-null  object        
 10  FIRST_PAGE_NAME    124344 non-null  object        
 11  END_PAGE_NAME      124323 non-null  object        
 12  VISIT_LOGGED       124344 non-null  category      
 13  D_PAGE             124344 non-null  object  

#### Producing tables for post

In [10]:
pg1 \
    .pipe(lambda df: df[df['PLATFORM']=='Web']) \
    .loc[:,'D_PAGE_CHAP1'] \
    .value_counts() \
    .to_frame('Num. of pages') \
    .head(12)

,Num. of pages
email request,13856
login,5631
registration,4819
home,4703
error,4345
cities,2539
benefits,2451
offer,2359
requests,1927
booknow,1263


In [11]:
pg1 \
    .pipe(lambda df: df[df['PLATFORM']=='Web']) \
    .pipe(lambda df: df[df['D_PAGE_CHAP1']=='benefits']) \
    .loc[:,'D_PAGE_CHAP2'] \
    .value_counts() \
    .to_frame('Num. of pages') \
    .head(5)

,Num. of pages
all,1751
hotel,152
dining,139
shopping,88
golf,34


In [12]:
pg1 \
    .pipe(lambda df: df[df['PLATFORM']=='Web']) \
    .pipe(lambda df: df[df['D_PAGE_CHAP1']=='benefits']) \
    .pipe(lambda df: df[df['D_PAGE_CHAP2']=='hotel']) \
    .loc[:,'D_PAGE_CHAP3'] \
    .value_counts() \
    .to_frame('Num. of pages') \
    .head(5)

,Num. of pages
tokyo,4
mumbai,3
london,2
taichung,1
bali,1


#### Creating a function to be pipelined(i.e. chained) with other functions, to move column to the desired loc

In [13]:
def moving_col(df, col_name, col_name_insert_at, after=True):
    if isinstance(df, pd.DataFrame):
        loc = 0
        col = df.pop(col_name)
        if after:
            loc = df.columns.get_loc(col_name_insert_at) + 1
        else:
            loc = df.columns.get_loc(col_name_insert_at)
        df.insert(loc, col_name, col)
        return df
    else:
        return None

### Create customer journey

#### 1. Create page number in each visit sorted by event date/time of page viewed ascendingly

In [14]:
pg2 = \
    pg1[['VISITID','PAGEID','D_PAGE_CHAP1']] \
        .sort_values(['VISITID','PAGEID']) \
        .reset_index(drop=True) \
        .assign(PAGE_NUM=lambda df: df.groupby('VISITID').cumcount()+1)

In [15]:
def print_results(df):
    df_c = None
    if isinstance(df, pd.DataFrame):
        df_c = df.query('VISITID=="JF - Visa APAC Android - PROD_2019-12-02_000000000000001"') \
            .pipe(lambda df: df.loc[:,'D_PAGE_CHAP1':]).set_index('D_PAGE_CHAP1',drop=True).copy()
        return df_c
    else:
        return df_c

In [16]:
print_results(pg2)

,PAGE_NUM
D_PAGE_CHAP1,
home,1
benefits,2
benefits,3
benefits,4
benefits,5
make a request,6
email request,7
home,8
benefits,9


#### 2. Getting continuous page boolean markers: the start row of each sequence is 1, subsequent row with the same D_PAGE_CHAP1 is 0

In [17]:
pg2 = \
    pg2.assign(CONTINUOUS_PG_BOOL_MARKER=lambda df: \
        ((df['PAGE_NUM'] == 1) \
            | ((df['VISITID'] == df['VISITID'].shift(1)) \
                & (df['D_PAGE_CHAP1'] != df['D_PAGE_CHAP1'].shift(1)))) \
        .astype('int'))

In [18]:
print_results(pg2)

,PAGE_NUM,CONTINUOUS_PG_BOOL_MARKER
D_PAGE_CHAP1,,
home,1,1
benefits,2,1
benefits,3,0
benefits,4,0
benefits,5,0
make a request,6,1
email request,7,1
home,8,1
benefits,9,1


#### 3. Then, cumsum within each visit so that each member row (including the start row) of a sequence would be assigned with the same number but different to the number assigned to subsequent sequences

In [19]:
pg2 = pg2.assign(PG_SQUNC_GROUP_MARKER=lambda df: df['CONTINUOUS_PG_BOOL_MARKER'].cumsum())

In [20]:
print_results(pg2)

,PAGE_NUM,CONTINUOUS_PG_BOOL_MARKER,PG_SQUNC_GROUP_MARKER
D_PAGE_CHAP1,,,
home,1,1,1
benefits,2,1,2
benefits,3,0,2
benefits,4,0,2
benefits,5,0,2
make a request,6,1,3
email request,7,1,4
home,8,1,5
benefits,9,1,6


#### 4. Create rank by row within each group as marked by PG_SQUNC_GROUP_MARKER

In [21]:
pg2 = pg2.assign(PG_SQUNC_GROUP_RANK=lambda df: \
          df.groupby(['VISITID','PG_SQUNC_GROUP_MARKER'])['PG_SQUNC_GROUP_MARKER'] \
               .cumcount().astype('int')+1)

In [22]:
print_results(pg2)

,PAGE_NUM,CONTINUOUS_PG_BOOL_MARKER,PG_SQUNC_GROUP_MARKER,PG_SQUNC_GROUP_RANK
D_PAGE_CHAP1,,,,
home,1,1,1,1
benefits,2,1,2,1
benefits,3,0,2,2
benefits,4,0,2,3
benefits,5,0,2,4
make a request,6,1,3,1
email request,7,1,4,1
home,8,1,5,1
benefits,9,1,6,1


#### 5. Create group size of each sequence and assign the group size to all members of the sequence

In [23]:
pg2 = pg2.assign(PG_SQUNC_GROUP_SIZE=lambda df: \
          df.groupby(['VISITID','PG_SQUNC_GROUP_MARKER'])['PG_SQUNC_GROUP_MARKER'] \
              .transform('size'))

In [24]:
print_results(pg2)

,PAGE_NUM,CONTINUOUS_PG_BOOL_MARKER,PG_SQUNC_GROUP_MARKER,PG_SQUNC_GROUP_RANK,PG_SQUNC_GROUP_SIZE
D_PAGE_CHAP1,,,,,
home,1,1,1,1,1
benefits,2,1,2,1,4
benefits,3,0,2,2,4
benefits,4,0,2,3,4
benefits,5,0,2,4,4
make a request,6,1,3,1,1
email request,7,1,4,1,1
home,8,1,5,1,1
benefits,9,1,6,1,2


#### 6. Filtering rows to the ones which only have PG_SQUNC_GROUP_RANK as 1, PG_SQUNC_GROUP_RANK = 1 points to the first occurance of charpter 1 in a group of charpter 1's which are the same as the the first occurance; then create charpter 1 customer journey; and create step depth for each point in charpter 1 customer journey

In [25]:
pg_ch1_journey = \
    pg2.pipe(lambda df: df[df['PG_SQUNC_GROUP_RANK']==1]) \
        .assign(PG_SQUNC_GROUP_FIRST_OCCUR_NUM=lambda df: \
            df.groupby('VISITID').cumcount()+1) \
        .assign(CAT_STRING=lambda df: \
            '[' + df['PG_SQUNC_GROUP_FIRST_OCCUR_NUM'].astype('str') + '] ') \
        .assign(TO_FORM_LIST=lambda df: \
            df['CAT_STRING'].str.cat(df['D_PAGE_CHAP1'])) \
        .pipe(lambda df: df.groupby('VISITID')['TO_FORM_LIST'].agg(list) \
            .to_frame('CHAP1_JOURNEY') \
            .join(df.groupby('VISITID')['PG_SQUNC_GROUP_SIZE'].agg(list) \
                .to_frame('CHAP1_JOURNEY_STEP_DEPTH')))

In [26]:
pg_ch1_journey.reset_index(drop=True).head(5)

,CHAP1_JOURNEY,CHAP1_JOURNEY_STEP_DEPTH
0,"[[1] home, [2] benefits, [3] make a request, [4] email request, [5] home, [6] benefits, [7] home, [8] benefits, [9] make a request, [10] email request, [11] benefits, [12] make a request, [13] email request, [14] benefits, [15] make a request, [16] email request, [17] benefits, [18] cities, [19] offer, [20] cities]","[1, 4, 1, 1, 1, 2, 1, 3, 1, 1, 11, 1, 1, 1, 1, 1, 1, 2, 1, 1]"
1,"[[1] benefits, [2] contents, [3] benefits, [4] offer, [5] error, [6] offer, [7] home, [8] benefits, [9] offer, [10] error, [11] offer, [12] home]","[3, 1, 1, 3, 1, 2, 1, 1, 3, 1, 2, 1]"
2,"[[1] home, [2] login, [3] legal, [4] help, [5] registration, [6] help, [7] home, [8] help]","[1, 4, 2, 1, 2, 1, 1, 13]"
3,"[[1] home, [2] login, [3] legal, [4] registration, [5] legal, [6] help, [7] home, [8] help, [9] login, [10] help, [11] registration, [12] help]","[1, 3, 1, 2, 1, 1, 1, 5, 1, 1, 1, 3]"
4,"[[1] home, [2] login, [3] home, [4] benefits, [5] home, [6] highlights, [7] offer, [8] home, [9] cities, [10] offer, [11] cities, [12] offer, [13] home, [14] requests, [15] make a request, [16] concierge chat request, [17] requests, [18] make a request, [19] requests, [20] concierge chat request, [21] requests, [22] home, [23] login, [24] home, [25] make a request, [26] concierge chat request, [27] home]","[1, 1, 2, 1, 1, 1, 1, 1, 3, 2, 3, 7, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1]"
